In [1]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `K` was defined in the following phases: ['FCC_A1', 'FCC_L12'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `S` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `P` was defined in the following phases: ['FCC_A1', 'FCC_4SL', 'HCP_A3', 'HCP_4SL'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typec

In [5]:
from numpy import save

save('./DataSets/all_phases.npy', all_phases)

In [2]:
def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int(int_mask) & (1 << i)) > 0]

def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def parseArray(amounts_str):
    amounts = amounts_str.replace('[', '').replace(']', '')
    return np.array(amounts.split(','), dtype=np.float32)


In [3]:
import pandas as pd

df_origin = pd.read_csv('DataSets/main_test.csv',delimiter = ';')
df = df_origin[df_origin['Error'].isna()].reset_index(drop=True)
df = df.drop(['Error'], axis =1)
df = df.rename(columns={"T": "Temp"})
df['Components'] = df['Components'].apply(getComponents)
df['phases'] = df['phases'].apply(getPhases)
df['possible_phases'] = df['possible_phases'].apply(getPhases)
df['amounts'] = df['amounts'].apply(parseArray)
print(df.dtypes)
df

iter                 int64
Temp                 int64
amounts             object
Components          object
phases              object
ellapsed_time      float64
possible_phases     object
P                    int64
dtype: object


,iter,Temp,amounts,Components,phases,ellapsed_time,possible_phases,P
0,500,1991,"[0.106400184, 0.070539184, 0.09098721, 0.08310...","[CO, CR, MO, NI, TA, V, W, ZR]",[LIQUID],28.562051,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
1,501,1613,"[0.055818617, 0.064889625, 0.21049123, 0.10578...","[CO, CR, MO, NI, TA, V, W, ZR]",[],25.476817,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
2,502,1253,"[0.0770591, 0.054417454, 0.11515643, 0.1760729...","[CO, CR, MO, NI, TA, V, W, ZR]","[LAVES_C14, LAVES_C15, NIZR, PT2V]",25.641211,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
3,503,429,"[0.1490746, 0.3030639, 0.28587595, 0.050671652...","[CO, CR, MO, NI, TA, V, W, ZR]","[CO2ZR, COV3_A15, LAVES_C14, LAVES_C15, MU_PHA...",24.184063,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
4,504,1324,"[0.29517522, 0.29847562, 0.06297051, 0.1217284...","[CO, CR, MO, NI, TA, V, W, ZR]","[LAVES_C14, LIQUID, MU_PHASE, SIGMA]",24.452664,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
...,...,...,...,...,...,...,...,...
61,561,2395,"[0.23362176, 0.097969905, 0.21510376, 0.058926...","[CO, CR, MO, NI, TA, V, W, ZR]",[LIQUID],20.424273,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
62,562,964,"[0.31064555, 0.10366003, 0.25342798, 0.0845174...","[CO, CR, MO, NI, TA, V, W, ZR]","[ALTA_SIGMA, CO2ZR, LAVES_C14, LAVES_C15, MU_P...",20.409927,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
63,563,781,"[0.31956258, 0.33813018, 0.050149467, 0.082267...","[CO, CR, MO, NI, TA, V, W, ZR]","[ALTA_SIGMA, CO2ZR, LAVES_C14, LAVES_C15, NI7Z...",20.757847,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
64,564,2790,"[0.31481087, 0.121363506, 0.20535697, 0.101599...","[CO, CR, MO, NI, TA, V, W, ZR]",[LIQUID],20.774998,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325


In [15]:
from modules.GetGm import getGM


def getTimeGM(db, Components, all_phases, possible_phases, amounts,T):
    tic = time.perf_counter()
    gm = getGM(db, Components, all_phases, possible_phases, amounts,T)
    toc = time.perf_counter()
    return (toc-tic, gm)

df_origin.loc[:,'gm_temp'] = df.apply(lambda x: getTimeGM(db, x.Components, all_phases, x.possible_phases, x.amounts, x.Temp), axis=1)
df_origin.loc[:,'gm_time'] = df_origin.apply(lambda x: x.gm_temp[0], axis=1)
df_origin.loc[:,'GM'] = df_origin.apply(lambda x: x.gm_temp[1], axis=1)
df_origin = df_origin.drop(['gm_temp'], axis=1)
df_origin

,iter,T,amounts,Components,phases,ellapsed_time,possible_phases,P,Error,gm_time,GM
0,500,1991,"[0.10640018773112153, 0.07053918581248933, 0.0...",21427291728907264,6084483977492414823814226985991697806408637340...,28.562051,4388592592290526150044810091035369884206945205...,101325,NaN,15.099462,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,501,1613,"[0.05581861917409891, 0.06488962166692959, 0.2...",21427291728907264,0,25.476817,4388592592290526150044810091035369884206945205...,101325,NaN,18.342375,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,502,1253,"[0.07705909494463434, 0.054417454064442705, 0....",21427291728907264,6813845498726864658394108293986205049292797343...,25.641211,4388592592290526150044810091035369884206945205...,101325,NaN,18.503303,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,503,429,"[0.1490745941107491, 0.30306388978042353, 0.28...",21427291728907264,3642361157803544944038421687014756117706903781...,24.184063,4388592592290526150044810091035369884206945205...,101325,NaN,15.851577,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,504,1324,"[0.2951752171998711, 0.29847561421829943, 0.06...",21427291728907264,6748105731502757109685565502551083482578950923...,24.452664,4388592592290526150044810091035369884206945205...,101325,NaN,14.991735,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
...,...,...,...,...,...,...,...,...,...,...,...
61,561,2395,"[0.2336217603454453, 0.09796990091803362, 0.21...",21427291728907264,6084483977492414823814226985991697806408637340...,20.424273,4388592592290526150044810091035369884206945205...,101325,NaN,12.886167,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
62,562,964,"[0.3106455612434611, 0.1036600294019806, 0.253...",21427291728907264,1312298908242624779712767693388063304048490051...,20.409927,4388592592290526150044810091035369884206945205...,101325,NaN,13.871445,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
63,563,781,"[0.31956257402502974, 0.3381301638547741, 0.05...",21427291728907264,6748105731502757109685565502551083482578950923...,20.757847,4388592592290526150044810091035369884206945205...,101325,NaN,13.293872,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
64,564,2790,"[0.3148108761611583, 0.12136350620584303, 0.20...",21427291728907264,6084483977492414823814226985991697806408637340...,20.774998,4388592592290526150044810091035369884206945205...,101325,NaN,13.445008,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [16]:
df_origin.to_csv("./DataSets/parsedDataset.csv", sep=";")

In [13]:
df_new = df[4:5]
arr = df_new.apply(lambda x: getTimeGM(db, x.Components, all_phases, x.possible_phases, x.amounts, x.Temp), axis=1)[4]
np.array(arr[1])[~np.isnan(arr[1])]

array([-21646.00734104, -21223.01390686, -24512.36346665, -85601.39207101,
       -18957.43871335, -22337.38471283])